In [1]:
# Import all necessary libraries.
import os
import cv2
import sys
import json
import matplotlib.image as mpimg
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../')
sys.path.append(NOMEROFF_NET_DIR)

In [2]:
# Import license plate recognition tools.
from nomeroff_net import  OptionsDetector

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

Using TensorFlow backend.
W0524 18:37:04.413190 139812976022848 deprecation.py:237] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0524 18:37:04.575600 139812976022848 deprecation.py:506] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0524 18:37:04.692456 139812976022848 deprecation.py:506] From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/moving_averages.py:211: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead

In [3]:
def optionChecker(datasetDir, imgFormat="png", partSize=100):
    ann = "ann"
    img = "img"
    annDir = os.path.join(datasetDir, ann)
    imgDir = os.path.join(datasetDir, img)
    
    imgFnames        = []
    nottedRegions    = []
    nottedcount_lines = []
    nottedStates     = []
    zones            = []
    annFnames        = []
    annData          = []
    i = 0
    counter = Counter()
    for dir_name, subdir_list, file_list in os.walk(annDir):
        for fname in file_list:
            ann_path = os.path.join(annDir, fname)
            
            i += 1
            annFnames.append(ann_path)
            with open(ann_path) as jsonR:
                data = json.load(jsonR)
            img_name = data["name"]
            annData.append(data)
            
            img_path = os.path.join(imgDir, "{}.{}".format(img_name, imgFormat))
            zones.append(cv2.cvtColor(mpimg.imread(img_path), cv2.COLOR_RGB2BGR))
            
            imgFnames.append(img_path)
            nottedRegions.append(data["region_id"])
            nottedStates.append(data["state_id"])
            nottedcount_lines.append(data["count_lines"])
            if i >= partSize:
                # find standart
                region_ids, stateIds, count_lines = optionsDetector.predict(zones)
                for regionId, stateId, zone, nottedRegion, nottedState, imgFname, annFname, annItem, nottedCountL, countL in zip(region_ids, stateIds, zones, nottedRegions, nottedStates, imgFnames, annFnames, annData, count_lines, nottedcount_lines):
                    
                    # region
                    bad_region=False
                    if int(regionId) != int(nottedRegion):
                        bad_region=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['regionPredicted'] = int(regionId)
                        print("REGION NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(regionId))
                        print("ANNOTATED: {}".format(nottedRegion))
                        counter["BAD_REGION"] += 1
                    else:
                        counter["GOOD_REGION"] += 1
                        
                    # state  
                    bad_state=False
                    if int(stateId) != int(nottedState):
                        bad_state=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['statePredicted'] = int(stateId)
                        print("STATE NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(stateId))
                        print("ANNOTATED: {}".format(nottedState))
                        counter["BAD_STATE"] += 1
                    else:
                        counter["GOOD_STATE"] += 1
                    
                    # count
                    bad_count=False
                    if int(countL) != int(nottedCountL):
                        bad_count=True
                        if not('moderation' in annItem):
                            annItem['moderation'] = {}
                        annItem['moderation']['countPredicted'] = int(countL)
                        print("COUNT LINES NOT CORRECT IN {}".format(imgFname))
                        print("PREDICTED: {}".format(countL))
                        print("ANNOTATED: {}".format(nottedCountL))
                        counter["BAD_COUNT"] += 1
                    else:
                        counter["GOOD_COUNT"] += 1

                    if 'moderation' in annItem and (bad_region or bad_state or bad_count):
                        with open(annFname, "w") as jsonW:
                            annItem['moderation']['isModerated'] = 0
                            json.dump(annItem, jsonW)                    
                    
                imgFnames        = []
                nottedRegions    = []
                nottedcount_lines = []
                nottedStates     = []
                zones            = []
                annFnames        = []
                annData          = []
    print(counter)

In [8]:
valDir = os.path.join(NOMEROFF_NET_DIR, "datasets/lnr")
optionChecker(valDir)

Counter()


5.283333333333333